In [4]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = '/content/AB_NYC_2019.csv'
data = pd.read_csv(file_path)

# Data Integrity Check: Display basic information and check for inconsistencies
print("Data Information:")
print(data.info())

print("\nData Summary Statistics:")
print(data.describe())

# Missing Data Handling: Display missing values count
print("\nMissing Values Count:")
print(data.isnull().sum())

# Handling missing data
# Convert 'last_review' to datetime
if 'last_review' in data.columns:
    data['last_review'] = pd.to_datetime(data['last_review'], errors='coerce')

# Impute 'last_review' with the median date
if 'last_review' in data.columns:
    median_date = data['last_review'].median()
    data['last_review'].fillna(median_date, inplace=True)

# Fill other missing values (e.g., 'reviews_per_month') with mean or mode
if 'reviews_per_month' in data.columns:
    data['reviews_per_month'].fillna(data['reviews_per_month'].mean(), inplace=True)

# Duplicate Removal: Identify and remove duplicate records
print("\nDuplicate Records Count: ", data.duplicated().sum())
data.drop_duplicates(inplace=True)

# Standardization: Ensure consistent formatting and units
data.columns = [col.lower().strip() for col in data.columns]
for col in data.select_dtypes(include=['object']).columns:
    data[col] = data[col].str.lower().str.strip()

# Outlier Detection: Identify and handle outliers
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] < lower_bound) | (df[column] > upper_bound)]

numeric_columns = data.select_dtypes(include=[np.number]).columns
outliers = pd.DataFrame()
for col in numeric_columns:
    outliers = pd.concat([outliers, detect_outliers(data, col)])

print("\nOutliers Detected:")
print(outliers)

# Optionally handle outliers (e.g., remove or cap them)
for col in numeric_columns:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data[col] = np.where(data[col] < lower_bound, lower_bound, data[col])
    data[col] = np.where(data[col] > upper_bound, upper_bound, data[col])

# Save the cleaned dataset
cleaned_file_path = '/content/AB_NYC_2019.csv'
data.to_csv(cleaned_file_path, index=False)

print("\nData Cleaning Process Completed. Cleaned data saved to:", cleaned_file_path)


Data Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review   